In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

# Load credentials securely
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Connect to Postgres
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# See all rows if needed
pd.set_option('display.max_rows', None)

### Business Question:
Which Snap job categories offer both high salaries and high public engagement on Reddit?

Combining job postings and public sentiment helps identify roles where Snap is investing the most money and generating the most conversation.

In [2]:
query = text("""
-- Extract salaries from raw strings
WITH extracted_salaries AS (
    SELECT *,
        CAST(REGEXP_REPLACE(SPLIT_PART(salary, '-', 1), '[^0-9]', '', 'g') AS INT) AS min_salary,
        CAST(REGEXP_REPLACE(SPLIT_PART(salary, '-', 2), '[^0-9]', '', 'g') AS INT) AS max_salary
    FROM fact_indeed_jobs
    WHERE salary IS NOT NULL AND salary != ''
),

-- Tag job focus areas
job_tagged AS (
    SELECT *,
        CASE
            WHEN LOWER(job_title) LIKE '%machine learning%' OR LOWER(job_title) LIKE '%ai%' THEN 'ml_ai_roles'
            WHEN LOWER(job_title) LIKE '%marketing%' THEN 'marketing'
            WHEN LOWER(job_title) LIKE '%research%' THEN 'research'
            WHEN LOWER(job_title) LIKE '%product%' THEN 'product'
            WHEN LOWER(job_title) LIKE '%design%' THEN 'design'
            WHEN LOWER(job_title) LIKE '%data%' THEN 'data'
            ELSE 'other'
        END AS tag
    FROM extracted_salaries
),

-- Aggregate salary stats by tag
job_stats AS (
    SELECT
        tag,
        COUNT(*) AS job_count,
        ROUND(AVG(max_salary)) AS avg_max_salary
    FROM job_tagged
    GROUP BY tag
),

-- Reddit sentiment aggregates (based on job_category_tag)
reddit_tagged AS (
    SELECT 
        job_category_tag AS tag,
        COUNT(*) AS post_count,
        ROUND(AVG(score), 1) AS avg_score,
        ROUND(AVG(num_comments), 1) AS avg_comments
    FROM reddit_posts
    GROUP BY job_category_tag
),

-- Join Indeed + Reddit
joined_view AS (
    SELECT 
        j.tag,
        j.job_count,
        j.avg_max_salary,
        r.post_count,
        r.avg_score,
        r.avg_comments,
        ROUND(j.avg_max_salary * r.post_count * r.avg_score / 1000.0, 1) AS sentiment_heat_index,
        RANK() OVER (ORDER BY j.avg_max_salary * r.post_count * r.avg_score DESC) AS heat_rank
    FROM job_stats j
    LEFT JOIN reddit_tagged r ON LOWER(j.tag) = LOWER(r.tag)
)

SELECT * FROM joined_view
ORDER BY heat_rank;
""")


In [3]:
df_mix = pd.read_sql(query, engine)
df_mix

,tag,job_count,avg_max_salary,post_count,avg_score,avg_comments,sentiment_heat_index,heat_rank
0,ml_ai_roles,7,323143.0,1325,623.3,115.8,266874917.3,1
1,research,1,259000.0,15,521.7,55.1,2026804.5,2
2,marketing,1,235000.0,7,353.6,62.4,581672.0,3
3,data,1,270000.0,8,130.0,53.3,280800.0,4


**Insight:**  
Snap’s highest-paying job category — “AI/ML roles” with an average max salary of $323K — also dominates Reddit engagement. These roles received over **1,300 posts** with an **average score of 623** and **116 comments per post**, far exceeding all other categories. This strong alignment suggests Snap’s AI investments are not only substantial but also highly visible and well-received by the tech community. Other categories like “research” and “marketing” show decent compensation but far lower public traction, while “data” roles show almost no engagement despite strong pay.

**Recommendation:**  
Snap should double down on hiring and brand messaging for AI/ML roles, leveraging the clear enthusiasm in online communities. At the same time, Snap should investigate why high-paying but lower-engagement roles like “data” or “marketing” are underrepresented online — whether due to lack of visibility, weaker brand associations, or job descriptions that don’t resonate with tech audiences. Cross-functional campaigns or insider spotlights may help elevate those categories.

**Prediction:**  
Snap’s public identity in AI will likely continue to strengthen if current hiring and discussion trends hold. However, without similar visibility-building efforts in less-hyped departments, Snap risks developing hiring bottlenecks or leaving certain teams talent-starved — not because of compensation, but because no one’s talking about them.

----------------------------------------------------------------------------------------------------

### Business Question:
How does job seniority level relate to salary competitiveness and spread at Snap?

Understanding how salaries change across levels like "Senior", "Staff", and "Principal" helps gauge Snap’s investment in leadership.

In [4]:
from sqlalchemy import text
import pandas as pd

query = text("""
-- Step 1: Extract min/max salary from raw strings
WITH cleaned_salaries AS (
    SELECT *,
        CAST(REGEXP_REPLACE(SPLIT_PART(salary, '-', 1), '[^0-9]', '', 'g') AS INT) AS min_salary,
        CAST(REGEXP_REPLACE(SPLIT_PART(salary, '-', 2), '[^0-9]', '', 'g') AS INT) AS max_salary
    FROM fact_indeed_jobs
    WHERE salary IS NOT NULL AND salary != ''
),

-- Step 2: Assign seniority levels
labeled_roles AS (
    SELECT *,
        CASE
            WHEN LOWER(job_title) LIKE '%principal%' THEN 'Principal'
            WHEN LOWER(job_title) LIKE '%staff%' THEN 'Staff'
            WHEN LOWER(job_title) LIKE '%senior%' THEN 'Senior'
            ELSE 'Other'
        END AS seniority
    FROM cleaned_salaries
),

-- Step 3: Aggregate salary stats by seniority
seniority_aggregates AS (
    SELECT
        seniority,
        COUNT(*) AS role_count,
        ROUND(AVG(min_salary)) AS avg_min_salary,
        ROUND(AVG(max_salary)) AS avg_max_salary,
        ROUND(AVG(max_salary - min_salary)) AS avg_spread,
        MAX(max_salary - min_salary) AS max_spread,
        MIN(max_salary - min_salary) AS min_spread
    FROM labeled_roles
    GROUP BY seniority
),

-- Step 4: Use window function to compare avg max salary growth between levels
ranked_salary AS (
    SELECT *,
        RANK() OVER (ORDER BY avg_max_salary DESC) AS salary_rank,
        LAG(avg_max_salary) OVER (ORDER BY avg_max_salary) AS prev_avg_max_salary
    FROM seniority_aggregates
)

SELECT 
    seniority,
    role_count,
    avg_min_salary,
    avg_max_salary,
    avg_spread,
    max_spread,
    min_spread,
    ROUND(
        100.0 * (avg_max_salary - prev_avg_max_salary) / NULLIF(prev_avg_max_salary, 0),
        1
    ) AS percent_increase,
    salary_rank
FROM ranked_salary
ORDER BY salary_rank;
""")

df_q1 = pd.read_sql(query, engine)
pd.set_option("display.max_rows", None)
df_q1


,seniority,role_count,avg_min_salary,avg_max_salary,avg_spread,max_spread,min_spread,percent_increase,salary_rank
0,Principal,2,235000.0,414000.0,179000.0,179000,179000,20.7,1
1,Staff,2,195000.0,343000.0,148000.0,148000,148000,27.0,2
2,Senior,1,162000.0,270000.0,108000.0,108000,108000,8.7,3
3,Other,5,141000.0,248400.0,107400.0,135000,76000,NaN,4


In [7]:
df_q1.to_csv('indeed_report2.csv', index=False)

**Insight:**  
Snap’s **Staff-level roles** show the largest jump in average salary, rising **27% from Senior to Staff** ($270K → $343K). Principal roles continue upward to **$414K** with the **widest spread ($179K)**, suggesting more flexible or negotiable comp at the top. This tiered structure reflects Snap’s growing investment in technical leadership, but wide spreads may signal inconsistency in expectations or scope.

**Recommendation:**  
Snap should clearly define compensation bands and expectations at each leadership tier. Roles like Principal with large spreads may need tighter guidelines or communication to help candidates understand their value and performance targets.

**Prediction:**  
Without increased clarity, Snap risks slower closes for senior candidates or internal equity concerns. As leadership hiring accelerates—especially in AI/ML—structured comp practices will be key to attracting and retaining top talent.
